### Data Ingestion
### Zeru Zhou
### zeruzhou9@gmail.com
### Data Glacier Internship
### LISUM16


In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle

In [4]:
! cp /kaggle.json ~/.kaggle

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [11]:
! kaggle datasets download mkechinov/ecommerce-behavior-data-from-multi-category-store

100% 4.29G/4.29G [02:17<00:00, 37.5MB/s]
100% 4.29G/4.29G [02:17<00:00, 33.4MB/s]


In [26]:
! pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 37.4 MB/s eta 0:00:00


In [108]:
import pandas as pd
import ray
import yaml
import numpy as np 

In [13]:
! unzip /content/ecommerce-behavior-data-from-multi-category-store.zip

Archive:  /content/ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
! cp -r '/content/2019-Oct.csv' '/content/drive/MyDrive'

In [16]:
oct_df = pd.read_csv('/content/2019-Oct.csv')
oct_df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [28]:
ray_df = ray.data.read_csv('/content/2019-Oct.csv')

2023-01-07 01:39:12,578	INFO worker.py:1538 -- Started a local Ray instance.
2023-01-07 01:39:14,928	WARNING read_api.py:315 -- ⚠️  The blocks of this dataset are estimated to be 10.0x larger than the target block size of 512 MiB. This may lead to out-of-memory errors during processing. Consider reducing the size of input files or using `.repartition(n)` to increase the number of dataset blocks.


In [30]:
ray_df

2023-01-07 01:41:06,759	WARNING formatters.py:918 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets>=8` for rich notebook output.


### Methods like Modin Ray, Dask are faster than pandas because it allows you to use many cores while reading data

### Design python script to validate the column features and get information about dataset

In [96]:
%%writefile Testutility.py
import yaml 
import logging
import os
import pandas as pd
import re


def read_config(file):
  with open(file, 'r') as ymal:
    try:
      config = yaml.load(ymal, Loader=yaml.Loader)
      config['columns'] = list(config['columns'])
      return config
    except yaml.YAMLError as err:
      logging.error(err)


def get_information(df, filepath):
  my_dict = {'columns':0, 'rows':0, 'file size in Mb':0}
  my_dict['rows'] = df.shape[0]
  my_dict['columns'] = df.shape[1]
  my_dict['file size in Mb'] = os.path.getsize(filepath)/1024 ** 2
  my_df = pd.DataFrame(my_dict, index = [0])
  return my_df


def reg(str):
  return re.sub('[^A-Za-z0-9]+', '', str)


def column_validation(df, config):
  df_list = []
  for i in df.columns:
    df_list.append(reg(i))
  df_list = map(lambda x: x.lower(), df_list)
  df_list = sorted(df_list)
  config_list = []
  for i in config['columns']:
    config_list.append(reg(i))
  config_list = map(lambda x: x.lower(), config_list)
  config_list = sorted(config_list)
  if len(df_list) == len(config_list) and df_list == config_list:
    print('validation passed')
    return 1
  else:
    print('validation failed')
    mismatch1 = list(set(config_list).difference(df_list))
    print(f'The columns in YAML config file that not detected in the uploaded data are {mismatch1}')
    mismatch2 = list(set(df_list).difference(config_list))
    print(f'The columns in the uploaded data that not found in config file are {mismatch2}')
    return 0

Writing Testutility.py


### Write a YAML file for column validation, and make file implementations more flexible

In [37]:
yaml_dict = {'file_type': 'csv',
 'dataset_name': 'oct_df',
 'file_name': '2019-Oct',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'columns': oct_df.columns}

In [39]:
with open('config.yaml', 'w') as ym:
  yaml.dump(yaml_dict, ym)

In [49]:
with open('/content/config.yaml', 'r') as ymal:
  config = yaml.load(ymal, Loader=yaml.Loader)

In [50]:
config

{'columns': Index(['event_time', 'event_type', 'product_id', 'category_id',
        'category_code', 'brand', 'price', 'user_id', 'user_session'],
       dtype='object'),
 'dataset_name': 'oct_df',
 'file_name': '2019-Oct',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|'}

In [53]:
config['columns'] = list(config['columns'])

In [54]:
config['columns']

['event_time',
 'event_type',
 'product_id',
 'category_id',
 'category_code',
 'brand',
 'price',
 'user_id',
 'user_session']

### Validate number of columns and column name, and extract summary using utility script

In [97]:
import Testutility

In [98]:
config = Testutility.read_config('/content/config.yaml')
config

{'columns': ['event_time',
  'event_type',
  'product_id',
  'category_id',
  'category_code',
  'brand',
  'price',
  'user_id',
  'user_session'],
 'dataset_name': 'oct_df',
 'file_name': '2019-Oct',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|'}

### Here is the summary table

In [99]:
Testutility.get_information(oct_df, '/content/2019-Oct.csv')

,columns,rows,file size in Mb
0,9,42448764,5406.010489


### Here is the Validation regarding columns

In [100]:
Testutility.column_validation(oct_df, config)

validation passed


1

### To ensure our code is successful, let's modify the dataframe a little bit

In [101]:
mod_df = oct_df

In [105]:
mod_df.columns = ['Hello', 'World', 'product_id', 'category_id',
       'category_code', 'brand', 'price', 'user_id', 'user_session']

In [106]:
Testutility.column_validation(mod_df, config)

validation failed
The columns in YAML config file that not detected in the uploaded data are ['eventtime', 'eventtype']
The columns in the uploaded data that not found in config file are ['hello', 'world']


0

### Hence, our validation function works well

### Now let's write the file into Pipe delimited file

In [ ]:
np.savetxt('Oct.csv', oct_df, delimiter='|')